In [ ]:
%pip install opencv-python

In [1]:
import cv2
import os
import json

In [42]:
video_path = "./assets/curved_wrist/truong.mp4"
save_dir = "extracted_segments"
os.makedirs(save_dir, exist_ok=True)

In [43]:
exercise_name = "bench"
error_name = "curved_wrist"
os.makedirs(os.path.join(save_dir, exercise_name, error_name), exist_ok=True)

# Tạo tên folder sẽ lưu từng segment
dest_folder = os.path.join(save_dir, exercise_name, error_name, "truong")
os.makedirs(dest_folder, exist_ok=True)

In [ ]:
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Cannot open video file.")

# Thông tin video
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_delay = int(400 / fps)  # Đơn vị: milliseconds

print(f"FPS: {fps}")

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"H264")

start_frame = None
end_frame = None
segment_count = 0
current_writer = None

# Danh sách metadata để lưu thông tin các segment
metadata = []
frames_buffer = []

display_width = 400  # Kích thước hiển thị video
display_height = int(height * (display_width / width))

# Đọc và hiển thị khung hình đầu tiên
ret, first_frame = cap.read()
if ret:
    resized_frame = cv2.resize(first_frame, (display_width, display_height))
    cv2.imshow("Video", resized_frame)
    print("Press any to start playing the video...")
    cv2.waitKey(0)
else:
    print("Error: Cannot read the first frame.")
    cap.release()
    cv2.destroyAllWindows()
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

    # Thay đổi kích thước hiển thị video
    resized_frame = cv2.resize(frame, (display_width, display_height))

    # Hiển thị video
    cv2.imshow("Video", resized_frame)

    key = cv2.waitKey(frame_delay) & 0xFF

    # Nhấn 's' để bắt đầu collect khung hình
    if key == ord("s"):
        if not frames_buffer:
            print(f"Frame start: {current_frame}")

    # Lưu khung hình vào bộ đệm nếu đang collect
    if len(frames_buffer) > 0 or key == ord("s"):
        frames_buffer.append(frame)

    # Nhấn 'e' để kết thúc collect và lưu đoạn video
    if key == ord("e") and frames_buffer:
        print(f"Frame end: {current_frame}. Please wait while saving the segment...")
        segment_path = os.path.join(dest_folder, f"segment_{segment_count}.mp4")
        out = cv2.VideoWriter(segment_path, fourcc, fps, (width, height))

        for buffered_frame in frames_buffer:
            out.write(buffered_frame)

        out.release()

        # Lưu thông tin metadata
        start_frame = current_frame - len(frames_buffer)
        end_frame = current_frame
        metadata.append(
            {
                "segment_path": segment_path,
                "video_src_path": video_path,
                "start_frame": start_frame,
                "end_frame": end_frame,
                "start_time_seconds": round(start_frame / fps, 2),
                "end_time_seconds": round(end_frame / fps, 2),
            }
        )

        # Reset bộ đệm
        frames_buffer = []
        segment_count += 1

        print("-----> Paused. Press any key to continue...")
        key = cv2.waitKey(0)

    # Nhấn 'q' để thoát
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

# Ghi metadata ra file JSON
metadata_path = os.path.join(dest_folder, "metadata.json")
with open(metadata_path, "w") as f:
    json.dump(metadata, f, indent=4)

print(f"Metadata saved to {metadata_path}")

### Compress video with ffmpeg

Install here: https://ffmpeg.org/download.html#build-windows


In [46]:
import subprocess

def compress_video(input_path, output_path):
    # Sử dụng ffmpeg để nén video với H264 codec
    command = [
        "ffmpeg",
        "-i",
        input_path,  # Đường dẫn tới video đầu vào
        "-vcodec",
        "libx264",  # Codec H264
        "-crf",
        "23",  # Chất lượng video
        "-preset",
        "fast",  # Preset nén
        "-y",  # Ghi đè nếu file đã tồn tại
        output_path,
    ]
    subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

In [ ]:
# Compress tất cả các segment đã lưu trong folder dest_folder
# Lấy danh sách các segment
segments = [f for f in os.listdir(dest_folder) if f.endswith(".mp4")]
segments

In [ ]:
import os
from concurrent.futures import ThreadPoolExecutor


def compress_segment(segment):
    segment_path = os.path.join(dest_folder, segment)
    compressed_path = os.path.join(dest_folder, f"compressed_{segment}")
    print(f"Compressing {segment}...")

    compress_video(segment_path, compressed_path)

    print(f"Compressed video saved to {compressed_path}")
    return compressed_path


# Sử dụng ThreadPoolExecutor để chạy nén song song
with ThreadPoolExecutor(
    max_workers=4
) as executor:  # Điều chỉnh số worker theo số CPU hoặc tài nguyên hệ thống
    futures = [executor.submit(compress_segment, segment) for segment in segments]

    for future in futures:
        try:
            compressed_video_path = future.result()
            print(f"Done: {compressed_video_path}")
        except Exception as e:
            print(f"Error compressing video: {e}")

Remove uncompressed files (or manually remove)


In [ ]:
segments

In [ ]:
# Remove uncompressed segments
for segment in segments:
    if segment.startswith("compressed_"):
        continue
    segment_path = os.path.join(dest_folder, segment)
    print(f"Removing {segment}...")
    os.remove(segment_path)

Double check the compressed before running the below cell


In [50]:
# Đổi tên các segment đã nén thành tên gốc
for segment in segments:
    compressed_path = os.path.join(dest_folder, "compressed_" + segment)
    original_path = os.path.join(dest_folder, segment)
    os.rename(compressed_path, original_path)